# Generative Adversarial Network



### Loading datset and preprocessing

In [8]:
from tensorflow.keras.datasets import mnist

# load mnist x_train
(x_train, _), (_, _) = mnist.load_data()

In [9]:
import numpy as np

""" Preprocessing """

# create batch_dim
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)

# convert to keras compatible float
x_train = x_train.astype(np.float32)

# normalization to inputs between [-1, 1]
MAX = np.max(x_train)
x_train = (x_train / (MAX/2)) - 1

print(f"Data_shape: {x_train.shape} - Input_range: [{np.min(x_train)}, {np.max(x_train)}]")

Data_shape: (60000, 28, 28, 1) - Input_range: [-1.0, 1.0]


### Discriminator Network


In [21]:
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

img_shape = x_train.shape[1:] # (28, 28, 1)

DISCRIMINATOR = Sequential()
DISCRIMINATOR.add(Flatten(input_shape=img_shape))
DISCRIMINATOR.add(Dense(512))
DISCRIMINATOR.add(LeakyReLU(alpha=0.2)) # alpha -> negative input * alpha
DISCRIMINATOR.add(Dense(256))
DISCRIMINATOR.add(LeakyReLU(alpha=0.2))
DISCRIMINATOR.add(Dense(1))
DISCRIMINATOR.add(Activation("sigmoid")) # sigmoid range [0,1] 0 - fake IMG, 1 real IMG

d_input = Input(shape=img_shape) # image in
d_pred = DISCRIMINATOR(d_input) # prediction in % out

DISCRIMINATOR = Model(inputs=d_input, outputs=d_pred)
DISCRIMINATOR.compile(
    loss="binary_crossentropy",
    optimizer=Adam(learning_rate=0.0002, beta_1=0.5),
    metrics=["accuracy"]
)

### Generator Net

In [22]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Reshape

z_dim = 100 # Noise Vector size

GENERATOR = Sequential()
GENERATOR.add(Dense(units=256, input_dim=z_dim))
GENERATOR.add(LeakyReLU(alpha=0.2))
GENERATOR.add(BatchNormalization(momentum=0.8))
GENERATOR.add(Dense(512))
GENERATOR.add(LeakyReLU(alpha=0.2))
GENERATOR.add(BatchNormalization(momentum=0.8))
GENERATOR.add(Dense(1024))
GENERATOR.add(LeakyReLU(alpha=0.2))
GENERATOR.add(BatchNormalization(momentum=0.8))
GENERATOR.add(Dense(np.prod(img_shape)))
GENERATOR.add(Activation("tanh"))
GENERATOR.add(Reshape(target_shape=img_shape))

noise = Input(shape=(z_dim,)) # noise in
img = GENERATOR(noise) # generated image out

GENERATOR = Model(inputs=noise, outputs=img)
# GENERATOR isn't compiled, because of GAN Net

### GAN Net

In [24]:
# combining DISCRIMINATOR and GENERATOR to GAN
noise_in = Input(shape=(z_dim,)) # noise in
img = GENERATOR(noise_in) # generated image
DISCRIMINATOR.trainable = False
d_pred = DISCRIMINATOR(img) # prediction if image input is real or fake
GAN = Model(inputs=noise_in, outputs=d_pred)

GAN.compile(
    loss="binary_crossentropy",
    optimizer=Adam(learning_rate=0.0002, beta_1=0.5)
)

### Training

In [28]:
import matplotlib.pyplot as plt
import os

PATH = "generated_images"
try:
    os.makedirs(PATH)
except FileExistsError:
    pass

# creating labels for dataset
batch_size = 32
y_real = np.ones(shape=(batch_size, 1)) # 1 = 100% real
y_fake = np.zeros(shape=(batch_size, 1)) # 0 = 0% real

epochs=10_000

for epoch in range(epochs):
    # get random images from real dataset
    rand_idxs = np.random.randint(low=0, high=x_train.shape[0], size=batch_size)
    real_imgs = x_train[rand_idxs]
    # generate fake images for training
    noise = np.random.normal(loc=0.0, scale=1.0, size=(batch_size, z_dim)) # noise vector (32, 100)
    fake_imgs = GENERATOR(noise, training=False)
    # train DISCRIMINATOR
    d_loss_real = DISCRIMINATOR.train_on_batch(real_imgs, y_real)
    d_loss_fake = DISCRIMINATOR.train_on_batch(fake_imgs, y_fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    # train GENERATOR
    noise = np.random.normal(loc=0.0, scale=1.0, size=(batch_size, z_dim))
    g_loss = GAN.train_on_batch(noise, y_real)

    # saving every 1000 steps
    if(epoch+1 % 1000) == 0:
        print(
            f"{epoch} - D_loss: {round(d_loss[0], 4)}"
            f" D_acc: {round(d_loss[1], 4)}"
            f" G_loss: {round(g_loss, 4)}"
        )

        rows, cols = 5, 5
        gen_imgs = GENERATOR.predict(noise) # generating image
        gen_imgs = 0.5 * gen_imgs + 0.5
        fig, axs = plt.subplots(rows, cols)
        cnt = 0
        for i in range(rows):
            for j in range(cols):
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap="gray")
                axs[i, j].axis("off")
                cnt +=1
        img_name = f"{epoch}.png"
        fig.savefig(os.path.join(PATH, img_name))
        plt.close()

0 - D_loss: 0.5937 D_acc: 0.6953 G_loss: 0.9309
1000 - D_loss: 0.5399 D_acc: 0.8125 G_loss: 1.0303
2000 - D_loss: 0.6389 D_acc: 0.6094 G_loss: 0.9617
3000 - D_loss: 0.5385 D_acc: 0.7344 G_loss: 1.0405
4000 - D_loss: 0.6456 D_acc: 0.6719 G_loss: 1.1038
5000 - D_loss: 0.7483 D_acc: 0.4844 G_loss: 0.9292
6000 - D_loss: 0.5971 D_acc: 0.6562 G_loss: 1.0329
7000 - D_loss: 0.6641 D_acc: 0.5938 G_loss: 1.0392
8000 - D_loss: 0.6272 D_acc: 0.7188 G_loss: 1.0679
9000 - D_loss: 0.6895 D_acc: 0.5156 G_loss: 1.0182
